In [1]:
!pip install langchain-text-splitters
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 747.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.8/401.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 11.1 MB/s eta 0:00:00


# load nghị định 100-2019 NĐ-CP

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [4]:
file_path = '/content/Nghị định-100-2019-NĐ-CP.docx.txt'
with open(file_path, 'r') as file:
            content = file.read()

# split content to chunk

In [5]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)

In [6]:
chunks_ = text_splitter.create_documents([content])
chunks = [c.page_content for c in chunks_]
print(f"The text has been broken down in {len(chunks)} chunks.")

The text has been broken down in 825 chunks.


# convert chunk to vector
I use dangvantuan/vietnamese-embedding model for embedding model.

In [7]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim

sentences = ['That is a happy person', 'That is a very happy person']

embedding_model = SentenceTransformer('dangvantuan/vietnamese-embedding').cuda() #('thenlper/gte-large').cuda()
embeddings = embedding_model.encode(sentences)
print(cos_sim(embeddings[0], embeddings[1]))

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/171 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.63k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/965 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

tensor([[0.9351]])


In [8]:
embeddings.shape

(2, 768)

In [9]:
import torch
import numpy as np

embeddings_ = []
batch_size = 16

for _id in range(0, len(chunks)//batch_size + int(len(chunks) % batch_size != 0)):
    start_id = _id * batch_size
    end_id = min(len(chunks), start_id + batch_size)
    print(start_id, end_id)
    batch_chunks = chunks[start_id:end_id]
    with torch.no_grad():
        embeddings = embedding_model.encode(batch_chunks)
    embeddings_.append(embeddings)

embeddings = torch.tensor(np.concatenate(embeddings_, 0))
embeddings.shape

0 16
16 32
32 48
48 64
64 80
80 96
96 112
112 128
128 144
144 160
160 176
176 192
192 208
208 224
224 240
240 256
256 272
272 288
288 304
304 320
320 336
336 352
352 368
368 384
384 400
400 416
416 432
432 448
448 464
464 480
480 496
496 512
512 528
528 544
544 560
560 576
576 592
592 608
608 624
624 640
640 656
656 672
672 688
688 704
704 720
720 736
736 752
752 768
768 784
784 800
800 816
816 825


torch.Size([825, 768])

# load LLM model
I use Qwen 2 7B Instruct

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
device = "cuda" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2-7B-Instruct",
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-7B-Instruct")

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/27.8k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

# convert question to embedding

In [10]:
question = 'vi phạm nồng độ cồn bị phạt bao nhiêu tiền?' #'uống rượu bia khi tham gia giao thông có bị phạt tiền không?'
query_embedding = embedding_model.encode([question])
query_embedding = torch.tensor(query_embedding)
query_embedding.shape

torch.Size([1, 768])

# calulate top 20 similar chunk by cosine

In [11]:
cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)
scores = cos(query_embedding.cuda(), embeddings.cuda())

In [12]:
indices = scores.topk(20).indices.cpu().long().numpy()
indices

array([650, 664, 665, 649, 102,  69,  75,  74, 134, 136, 151, 226, 262,
       252, 260, 314, 767, 418, 236, 648])

In [13]:
import numpy as np
chunks_ = np.array(chunks)
chunks_[indices]

array(['2. Phạt từ 4.000.000 đồng đến 6.000.000 đồng đối với hành vi khi làm nhiệm vụ mà trong máu hoặc hơi thở có nồng độ cồn vượt quá 50 miligam đến 80 miligam/100 mililít máu hoặc vượt quá 0,25 miligam đến 0,4 miligam/1 lít khí thở.\n3. Phạt từ 6.000.000 đồng đến 8.000.000 đồng đối với một trong các hành vi vi phạm sau đây:\na) Khi làm nhiệm vụ mà trong máu hoặc hơi thở có nồng độ cồn vượt quá 80 miligam/100 mililít máu hoặc vượt quá 0,4 miligam/1 lít khí thở;',
       'b) Khi làm nhiệm vụ mà trong máu hoặc hơi thở có nồng độ cồn nhưng chưa vượt quá 50 miligam/100 mililít máu hoặc chưa vượt quá 0,25 miligam/1 lít khí thở.\n6. Phạt tiền từ 16.000.000 đồng đến 18.000.000 đồng đối với hành vi khi làm nhiệm vụ mà trong máu hoặc hơi thở có nồng độ cồn vượt quá 50 miligam đến 80 miligam/100 mililít máu hoặc vượt quá 0,25 miligam đến 0,4 miligam/1 lít khí thở.\n7. Phạt tiền từ 30.000.000 đồng đến 40.000.000 đồng đối với một trong các hành vi vi phạm sau đây:',
       'a) Khi làm nhiệm vụ m

# create question prompt

In [14]:
Contexts = "\n".join(chunks_[indices].tolist())
print(Contexts)

2. Phạt từ 4.000.000 đồng đến 6.000.000 đồng đối với hành vi khi làm nhiệm vụ mà trong máu hoặc hơi thở có nồng độ cồn vượt quá 50 miligam đến 80 miligam/100 mililít máu hoặc vượt quá 0,25 miligam đến 0,4 miligam/1 lít khí thở.
3. Phạt từ 6.000.000 đồng đến 8.000.000 đồng đối với một trong các hành vi vi phạm sau đây:
a) Khi làm nhiệm vụ mà trong máu hoặc hơi thở có nồng độ cồn vượt quá 80 miligam/100 mililít máu hoặc vượt quá 0,4 miligam/1 lít khí thở;
b) Khi làm nhiệm vụ mà trong máu hoặc hơi thở có nồng độ cồn nhưng chưa vượt quá 50 miligam/100 mililít máu hoặc chưa vượt quá 0,25 miligam/1 lít khí thở.
6. Phạt tiền từ 16.000.000 đồng đến 18.000.000 đồng đối với hành vi khi làm nhiệm vụ mà trong máu hoặc hơi thở có nồng độ cồn vượt quá 50 miligam đến 80 miligam/100 mililít máu hoặc vượt quá 0,25 miligam đến 0,4 miligam/1 lít khí thở.
7. Phạt tiền từ 30.000.000 đồng đến 40.000.000 đồng đối với một trong các hành vi vi phạm sau đây:
a) Khi làm nhiệm vụ mà trong máu hoặc hơi thở có nồng

In [15]:
question_prompt = f'''Using the information contained in the context,
give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the number of the source document when relevant.
If the answer cannot be deduced from the context, do not give an answer.

Context:
{Contexts}

Now here is the question you need to answer.

Question: {question}'''

In [16]:
print(question_prompt)

Using the information contained in the context,
give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the number of the source document when relevant.
If the answer cannot be deduced from the context, do not give an answer.

Context:
2. Phạt từ 4.000.000 đồng đến 6.000.000 đồng đối với hành vi khi làm nhiệm vụ mà trong máu hoặc hơi thở có nồng độ cồn vượt quá 50 miligam đến 80 miligam/100 mililít máu hoặc vượt quá 0,25 miligam đến 0,4 miligam/1 lít khí thở.
3. Phạt từ 6.000.000 đồng đến 8.000.000 đồng đối với một trong các hành vi vi phạm sau đây:
a) Khi làm nhiệm vụ mà trong máu hoặc hơi thở có nồng độ cồn vượt quá 80 miligam/100 mililít máu hoặc vượt quá 0,4 miligam/1 lít khí thở;
b) Khi làm nhiệm vụ mà trong máu hoặc hơi thở có nồng độ cồn nhưng chưa vượt quá 50 miligam/100 mililít máu hoặc chưa vượt quá 0,25 miligam/1 lít khí thở.
6. Phạt tiền từ 16.000.000 đồng đến 18.000.000 đồng đối với h

# Ask Qwen2 7B

In [ ]:
device = "cuda" # the device to load the model onto

messages = [
    {"role": "system", "content": "You are a helpful assistant. Please answer question more detail base on context!"},
    {"role": "user", "content": question_prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(device)

generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Vi phạm với nồng độ cồn vượt quá 50 miligam đến 80 miligam/100 mililít máu hoặc vượt quá 0,25 miligam đến 0,4 miligam/1 lít khí thở thì bị phạt từ 4.000.000 đồng đến 6.000.000 đồng. Nếu nồng độ cồn vượt quá 80 miligam/100 mililít máu hoặc vượt quá 0,4 miligam/1 lít khí thở thì bị phạt từ 6.000.000 đồng đến 8.000.000 đồng.


# Ask Qwen2 72B
Because I can't run it on Colab Free tier, I use Together AI to test Qwen2 72B

In [17]:
pip install together

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 4.1 MB/s eta 0:00:00


In [19]:
import os
from google.colab import userdata
from together import Together

client = Together(api_key=userdata.get('togetherai_api'))

response = client.chat.completions.create(
    model="Qwen/Qwen2-72B-Instruct",
    messages=[{"role": "user", "content": question_prompt}],
)
print(response.choices[0].message.content)

Cụ thể mức phạt tùy thuộc vào nồng độ cồn như sau:
- Từ 4.000.000 đồng đến 6.000.000 đồng nếu nồng độ cồn vượt quá 50 miligam đến 80 miligam/100 mililít máu hoặc 0,25 miligam đến 0,4 miligam/1 lít khí thở (2).
- Từ 6.000.000 đồng đến 8.000.000 đồng nếu nồng độ cồn vượt quá 80 miligam/100 mililít máu hoặc 0,4 miligam/1 lít khí thở (3a).
- Từ 16.000.000 đồng đến 18.000.000 đồng nếu nồng độ cồn vượt quá 50 miligam đến 80 miligam/100 mililít máu hoặc 0,25 miligam đến 0,4 miligam/1 lít khí thở (6).
- Từ 30.000.000 đồng đến 40.000.000 đồng nếu nồng độ cồn vượt quá 80 miligam/100 mililít máu hoặc 0,4 miligam/1 lít khí thở (7a).


# Ask Llama 3.1 405B
Because I can't run it on Colab Free tier, I use Together AI to test Llama 3.1 405B

In [20]:
import os
from google.colab import userdata
from together import Together

client = Together(api_key=userdata.get('togetherai_api'))

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-405B-Instruct-Turbo",
    messages=[{"role": "user", "content": question_prompt}],
)
print(response.choices[0].message.content)

Mức phạt tiền đối với hành vi vi phạm nồng độ cồn có thể khác nhau tùy thuộc vào mức độ vi phạm và loại phương tiện. Dưới đây là một số mức phạt tiền được quy định trong văn bản:

- Đối với người điều khiển xe ô tô, máy kéo, các loại xe tương tự xe ô tô:
 + Nồng độ cồn chưa vượt quá 50 miligam/100 mililít máu hoặc chưa vượt quá 0,25 miligam/1 lít khí thở: phạt tiền từ 6.000.000 đồng đến 8.000.000 đồng.
 + Nồng độ cồn vượt quá 50 miligam đến 80 miligam/100 mililít máu hoặc vượt quá 0,25 miligam đến 0,4 miligam/1 lít khí thở: phạt tiền từ 10.000.000 đồng đến 12.000.000 đồng hoặc từ 30.000.000 đồng đến 40.000.000 đồng.
 + Nồng độ cồn vượt quá 80 miligam/100 mililít máu hoặc vượt quá 0,4 miligam/1 lít khí thở: phạt tiền từ 30.000.000 đồng đến 40.000.000 đồng.

- Đối với người điều khiển xe mô tô hai bánh, xe mô tô ba bánh:
 + Nồng độ cồn chưa vượt quá 50 miligam/100 mililít máu hoặc chưa vượt quá 0,25 miligam/1 lít khí thở: phạt tiền từ 2.000.000 đồng đến 4.000.000 đồng.
 + Nồng độ cồn vượ

# The result base on LLM model and embedding model.
## We can see that larger and better LLM will answer better. But because the relevant chunk don't show enough contexts than LLm can't answer correctly!

## We need to rerank relevant chunk after get top 20 similar chunk to improve result

## I just use basic chunking strategy, maybe advanced chunking can help improve RAG QA system (late chunking, semantic chunking) or use longer context embedding model

## Spelling errors in the text may occur when downloading as a .txt file. Perhaps we need to clean the data for better results.